In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import floor
from datetime import date, timedelta

#for archiving
import os 
import argparse
import seaborn as sns
import pickle
import sklearn
import math
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
#for reporting 
#import perf
#from pandas.plotting import register_matplotlib_conve
import shutil
import calendar
from datetime import datetime
from fpdf import FPDF
from matplotlib import rcParams
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False

In [2]:
protokoll_path = r"01_Results/pred_day1.csv"
pred_day1 = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"01_Results/pred_day2.csv"
pred_day2 = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"01_Results/pred_day3.csv"
pred_day3 = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"01_Results/pred_day4.csv"
pred_day4 = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"01_Results/pred_day5.csv"
pred_day5 = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"01_Results/pred_day6.csv"
pred_day6 = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"01_Results/pred_day7.csv"
pred_day7 = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"01_Results/pred_naive_oneDay.csv"
pred_naive = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

protokoll_path = r"00_Daten/batchzahlen_gesamt.csv"
batchzahlen_gesamt = pd.read_csv(
    protokoll_path, encoding='unicode_escape', sep=";", error_bad_lines=True)

pred_day1 = pred_day1[["true_0","pred_0","lower_0","upper_0"]]
pred_day2 = pred_day2[["true_1","pred_1","lower_1","upper_1"]]
pred_day3 = pred_day3[["true_2","pred_2","lower_2","upper_2"]]
pred_day4 = pred_day4[["true_3","pred_3","lower_3","upper_3"]]
pred_day5 = pred_day5[["true_4","pred_4","lower_4","upper_4"]]
pred_day6 = pred_day6[["true_5","pred_5","lower_5","upper_5"]]
pred_day7 = pred_day7[["true_6","pred_6","lower_6","upper_6"]]


pred_naive = pred_naive[["true_naive","pred_naive"]]

batchzahlen_gesamt = batchzahlen_gesamt[batchzahlen_gesamt["Lager"] == "DPS"]
batchzahlen_gesamt["ï»¿Datum"] = pd.to_datetime(batchzahlen_gesamt["ï»¿Datum"])
batchzahlen_gesamt['day_of_week'] = batchzahlen_gesamt['ï»¿Datum'].dt.day_name()
batchzahlen_gesamt = batchzahlen_gesamt[["ï»¿Datum","day_of_week" ,"VPE_Gesamt", "Lager"]]
batchzahlen_gesamt.columns = ["Datum","Wochentag", "Bestellzahlen_Gesamt", "Lager"]

batchzahlen_gesamt['Wochentag'] = batchzahlen_gesamt['Wochentag'].replace(['Monday'],'Montag')
batchzahlen_gesamt['Wochentag'] = batchzahlen_gesamt['Wochentag'].replace(['Tuesday'],'Dienstag')
batchzahlen_gesamt['Wochentag'] = batchzahlen_gesamt['Wochentag'].replace(['Wednesday'],'Mittwoch')
batchzahlen_gesamt['Wochentag'] = batchzahlen_gesamt['Wochentag'].replace(['Thursday'],'Donnerstag')
batchzahlen_gesamt['Wochentag'] = batchzahlen_gesamt['Wochentag'].replace(['Friday'],'Freitag')
batchzahlen_gesamt['Wochentag'] = batchzahlen_gesamt['Wochentag'].replace(['Saturday'],'Samstag')
batchzahlen_gesamt['Wochentag'] = batchzahlen_gesamt['Wochentag'].replace(['Sunday'],'Sonntag')

import pickle
a_file = open("pred.pkl", "rb")
pred_dic = pickle.load(a_file)
a_file.close()

# Report on Test Set

In [3]:
def merge_pre_real(key):
    true = list(map(int, pred_dic[key]["true"]))
    pred = list(map(int, pred_dic[key]["pred"]))
    lower_bound = list(map(int, pred_dic[key]["lower"]))
    upper_bound = list(map(int, pred_dic[key]["upper"]))
    true_pred = batchzahlen_gesamt[batchzahlen_gesamt.Bestellzahlen_Gesamt.isin(
    pred_dic[key]["true"])]

    data = {
        "true": true,
        "prediction": pred,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound
    }
    df = pd.DataFrame(data)
    #sollte es mehrfach die exakt gleiche Bestellmenge geben 
    while(len(true_pred["Datum"]) >= 7):
        if len(true_pred["Datum"]) >= 7:
            delete_value = true_pred.reset_index()
            mean = delete_value["index"].mean()
            maxx = delete_value["index"].max() - mean
            maxx = maxx * maxx
            minn = delete_value["index"].min() - mean
            minn = minn * minn
            if maxx >= minn:
                true_pred = true_pred.drop(index=delete_value["index"].max())
            else:
                true_pred = true_pred.drop(index=delete_value["index"].min())


    overview_pred = true_pred.merge(df, how='left', left_on="Bestellzahlen_Gesamt", right_on="true")

    overview_pred = overview_pred[["Datum", "Wochentag", "Bestellzahlen_Gesamt",
                               "prediction", "lower_bound", "upper_bound", "Lager"]]
    
    return overview_pred

In [4]:
def get_barchart(overview_pred, filename):
    sns.set_context('talk')
    # Create a grouped bar chart, with job as the x-axis
    # and gender as the variable we're grouping on so there
    # are two bars per job.
    fig, ax = plt.subplots(figsize=(12, 8))
    # Our x-axis. We basically just want a list
    # of numbers from zero with a value for each
    # of our jobs.
    x = np.arange(len(overview_pred["Wochentag"]))
    # Define bar width. We need this to offset the second bar.
    bar_width = 0.4
    b1 = ax.bar(x, overview_pred["Bestellzahlen_Gesamt"], width=bar_width, label='Realwert')
    # Same thing, but offset the x.
    b2 = ax.bar(x + bar_width, overview_pred["prediction"], width=bar_width, label='Prognose')
    # Fix the x-axes.
    ax.set_xticks(x + bar_width / 2)
    ax.set_xticklabels(overview_pred["Wochentag"])
    #textstr = f"{min(overview_pred['Datum']).date()} - {max(overview_pred['Datum']).date()}"
    # these are matplotlib.patch.Patch properties
    #props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    # place a text box in upper left in axes coords
    #ax.text(0.37, 0.99, textstr, transform=ax.transAxes, fontsize=14,
        #verticalalignment='top', bbox=props)
    # Add legend.
    ax.legend(loc=1, prop={'size': 14})

    # Axis styling.
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_color('#DDDDDD')
    ax.tick_params(bottom=False, left=False)
    ax.set_axisbelow(True)
    ax.yaxis.grid(True, color='#EEEEEE')
    ax.xaxis.grid(False)

    # Add axis and chart labels.
    ax.set_xlabel('Wochentage', labelpad=15)
    ax.set_ylabel('Bestellzahlen', labelpad=15)
    #ax.set_title('Vergleich Prognose und Realwert', pad=15)

    # For each bar in the chart, add a text label.
    for bar in ax.patches:
        # The text annotation for each bar should be its height.
        bar_value = bar.get_height()
        # Format the text with commas to separate thousands. You can do
        # any type of formatting here though.
        text = f'{bar_value:,}'
        # This will give the middle of each bar on the x-axis.
        text_x = bar.get_x() + bar.get_width() / 2
        # get_y() is where the bar starts so we add the height to it.
        text_y = bar.get_y() + bar_value
        # If we want the text to be the same color as the bar, we can
        # get the color like so:
        bar_color = bar.get_facecolor()
        # If you want a consistent color, you can just set it as a constant, e.g. #222222
        ax.text(text_x, text_y, text, ha='center', va='bottom', color=bar_color,
          size=12)
    plt.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()
        #fig.tight_layout()

In [5]:
def get_linechart(overview_pred,filename):

    x = np.arange(len(overview_pred["Datum"]))
    y1 = overview_pred["Bestellzahlen_Gesamt"]
    y2 = overview_pred["prediction"]
    y3 = overview_pred["lower_bound"]
    y4 = overview_pred["upper_bound"]
    
    plt.subplots(figsize=(12, 8))
    plt.plot(x, y1, label = "Realerwert")
    plt.plot(x, y2, label = "Prognose")
    plt.plot(x, y3,label = "90% ",color='grey')
    plt.plot(x, y4,color='grey')
    plt.legend(loc=1, prop={'size': 14})
    plt.fill_between(x, y3, y4, color='grey', alpha=0.1)
    plt.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()
    #plt.show()

In [6]:
def get_numbers(true_value, prediction):
    pred_numbers = []
    real_numbers = []
    
    pred_mean = round(prediction.mean())
    pred_std = round(prediction.std())
    pred_05quantile = round(prediction.quantile(q=0.5))
    pred_numbers.append(pred_mean)
    pred_numbers.append(pred_std)
    pred_numbers.append(pred_05quantile)
    
    real_mean = round(true_value.mean())
    real_std = round(true_value.std())
    real_05quantile = round(true_value.quantile(q=0.5))
    real_numbers.append(real_mean)
    real_numbers.append(real_std)
    real_numbers.append(real_05quantile)

    return pred_numbers,real_numbers

def get_errors(true_value, prediction):

    mse = sklearn.metrics.mean_squared_error(true_value, prediction)
    mae = sklearn.metrics.mean_absolute_error(true_value, prediction)
    mape = sklearn.metrics.mean_absolute_percentage_error(true_value, prediction)
    
    mse = round(mse)
    mae = round(mae)
    mape = round(mape,2)
    
    return mse,mae,mape

In [7]:
class PDF(FPDF):
    def __init__(self):
        super().__init__()
        self.WIDTH = 210
        self.HEIGHT = 297
        
    def header(self):
        # Custom logo and positioning
        # Create an `assets` folder and put any wide and short image inside
        # Name the image `logo.png`
        self.image('00_assets/Budni_Logo.png', 10, 8, 33)
        self.set_font('Arial', 'B', 11)
        self.cell(self.WIDTH - 80)
        #self.cell(60, 1, 'DPS_Prognose', 0, 0, 'R')
        self.ln(20)
        
    def footer(self):
        # Page numbers in the footer
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')



In [8]:
def create_folder(PLOT_DIR):
    # Delete folder if exists and create it again
    try:
        shutil.rmtree(PLOT_DIR)
        os.mkdir(PLOT_DIR)
    except FileNotFoundError:
        os.mkdir(PLOT_DIR)

In [9]:
def get_report(number, PLOT_DIR,pdf):
    
    #Ordner für Charts erstellen
    create_folder(f"00_Report_Graphs/{PLOT_DIR}")
    key = f"info_{number}"
    #Merge Prognoe mit Echten Daten
    df_prognose = merge_pre_real(key)
    #Barchart erstellen und im Pfad ablegen
    get_barchart(df_prognose,f"00_Report_Graphs/{PLOT_DIR}/{number}_bar_info.png")
    #linechart erstellen und im Pfad ablegen
    get_linechart(df_prognose,f"00_Report_Graphs/{PLOT_DIR}/{number}_line_info.png")
    # Date-time zu Date 
    df_prognose.Datum = pd.to_datetime(df_prognose.Datum).dt.date
    #Standard Größe für PDF 
    pdf_w=210
    pdf_h=297
    #Größen für Charts 
    WIDTH = 95
    HEIGHT = 80
    
    pdf.add_page()
    pdf.set_author(author= "Lukas Dech")
    pdf.set_xy(0, 0)
    pdf.set_font('arial', 'B', 14)
    pdf.cell(60)
    pdf.cell(10,14,"",0,2)
    pdf.cell(105, 10, "Bestellzahlen Vorhersage Reporting DPS", 0, 2, 'C')
    pdf.cell(90, 3, " ", 0, 2, 'C')
    #pdf.set_font('arial',"", 12)
    pdf.cell(105, 8, f"{min(df_prognose['Datum'])} - {max(df_prognose['Datum'])}", 1, 2, 'C')
    pdf.cell(90, 15, " ", 0, 2, 'C')
    pdf.cell(-45)
    pdf.cell(30, 10, 'Datum', 1, 0, 'C')
    pdf.cell(30, 10, 'Wochentag', 1, 0, 'C')
    pdf.cell(30, 10, 'Realwert', 1, 0, 'C')
    pdf.cell(30, 10, 'Prognose', 1, 0, 'C')
    pdf.cell(30, 10, 'Lower', 1, 0, 'C')
    pdf.cell(30, 10, 'Upper', 1, 2, 'C')
    #pdf.set_x = 0

    pdf.cell(-150)
    pdf.set_font('arial', '', 12)
    for i in range(0, len(df_prognose)):
        pdf.cell(30, 8, '%s' % (df_prognose.Datum.iloc[i]), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(df_prognose.Wochentag.iloc[i])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(df_prognose.Bestellzahlen_Gesamt.iloc[i])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(df_prognose.prediction.iloc[i])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(df_prognose.lower_bound.iloc[i])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(df_prognose.upper_bound.iloc[i])), 1, 1, 'C')
        pdf.cell(5)
    pdf.cell(90, 20, " ", 0, 2, 'C')
    pdf.cell(-30)
    y = pdf.get_y()
    pdf.image(f"00_Report_Graphs/{PLOT_DIR}/{number}_bar_info.png", x = 5, y = None, w = WIDTH, h = HEIGHT, type = '', link = '')
    pdf.image(f"00_Report_Graphs/{PLOT_DIR}/{number}_line_info.png", x = WIDTH + 10, y = y, w = WIDTH, h = HEIGHT, type = '', link = '')
    pdf.cell(10,15,"",0,2)
    pdf.cell(45)
    pdf.set_font('arial', 'B', 12)
    pdf.cell(25, 10, 'Kennzahl', 1, 0, 'C')
    pdf.cell(25, 10, 'Realwert', 1, 0, 'C')
    pdf.cell(25, 10, 'Prognose', 1, 0, 'C')
    pdf.cell(15, 10, '', 0, 0, 'C')
    pdf.cell(25, 10, 'Errors', 1, 0, 'C')
    pdf.cell(25, 10, 'Prognose', 1, 1, 'C')
    pdf.set_font('arial',"", 12)
    
    mse,mae,mape = get_errors(df_prognose["Bestellzahlen_Gesamt"],df_prognose["prediction"])
    pred, real = get_numbers(df_prognose["Bestellzahlen_Gesamt"],df_prognose["prediction"])
    pdf.cell(20)
    pdf.cell(25, 10, 'MEAN', 1, 0, 'C')
    pdf.cell(25, 10, f'{real[0]}', 1, 0, 'C')
    pdf.cell(25, 10, f'{pred[0]}', 1, 0, 'C')
    pdf.cell(15, 10, '', 0, 0, 'C')
    pdf.cell(25, 10, 'MSE', 1, 0, 'C')
    pdf.cell(25, 10, f'{mse}', 1, 1, 'C')
    
    pdf.cell(20)
    pdf.cell(25, 10, 'STD', 1, 0, 'C')
    pdf.cell(25, 10, f'{real[1]}', 1, 0, 'C')
    pdf.cell(25, 10, f'{pred[1]}', 1, 0, 'C')
    pdf.cell(15, 10, '', 0, 0, 'C')
    pdf.cell(25, 10, 'MAE', 1, 0, 'C')
    pdf.cell(25, 10, f'{mae}', 1, 1, 'C')
    
    pdf.cell(20)
    pdf.cell(25, 10, 'Q0.5', 1, 0, 'C')
    pdf.cell(25, 10, f'{real[2]}', 1, 0, 'C')
    pdf.cell(25, 10, f'{pred[2]}', 1, 0, 'C')
    pdf.cell(15, 10, '', 0, 0, 'C')
    pdf.cell(25, 10, 'MAPE', 1, 0, 'C')
    pdf.cell(25, 10, f'{mape}', 1, 2, 'C')
    
    return pdf
    #pdf.output('test.pdf', 'F')

***

In [621]:
pdf = PDF()
for number in range(1,25):
    pdf = get_report(number,f"report_{number}_graph",pdf)
pdf.output('Report_DPS.pdf', 'F')


''

***

# Report on whole Time Series

In [24]:
def merge_PreDay_with_true_values(pred_day,batchzahlen_gesamt):
    column_names = pred_day.columns.to_list()
    pred_day.columns = ["true","pred","lower","upper"]
    pred_day["true"] = pred_day[["true"]].astype(int)
    pred_day["pred"] = pred_day[["pred"]].astype(int)
    pred_day["lower"] = pred_day[["lower"]].astype(int)
    pred_day["upper"] = pred_day[["upper"]].astype(int)

    only_since_2017 = batchzahlen_gesamt[batchzahlen_gesamt["Datum"] >= "2017-01-01" ]
    
    merged = only_since_2017.merge(pred_day, how='right', left_on="Bestellzahlen_Gesamt", right_on="true")
    
    merged = merged[["Datum","Wochentag","Bestellzahlen_Gesamt","true","pred","lower","upper"]]
    merged.columns = ["Datum","Wochentag","Bestellzahlen_Gesamt",column_names[0],column_names[1],column_names[2],column_names[3]]
    
    return merged

In [25]:
def errors_nach_day_prediction(days,true_index,pred_index):
    errors = []
    for df in days:
        err = get_errors(df.iloc[:,true_index],df.iloc[:,pred_index])
        errors.append(err)
    return errors

def numbers_nach_day_prediction(days,true_index,pred_index):
    pred_num = []
    real_num = []
    for df in days:
        pred, real = get_numbers(df.iloc[:,true_index],df.iloc[:,pred_index])
        #print(df["Wochentag"].iloc[3])
        pred_num.append(pred)
        real_num.append(real)
    return pred_num, real_num
        


In [26]:
def look_at_weekdays(days,weekday):
    array_weekdays = []
    #print(weekday_dfs)
    for df in days:
        df = merge_PreDay_with_true_values(days[0],batchzahlen_gesamt)
        df = df[df["Wochentag"] == weekday]
        array_weekdays.append(df)
    result = pd.concat(array_weekdays)
    result = result.drop_duplicates(keep="first")
    return result

In [27]:
# Errors nach Wochentag
#weekdays = ["Montag","Dienstag","Mittwoch","Donnerstag","Freitag", "Samstag"]
#weekday_array = []
#for weekday in weekdays: 
 #   day = look_at_weekdays(days,weekday)
  #  weekday_array.append(day)
#errors_nach_day_prediction(weekday_array,3,4)

In [28]:
def get_bar_per_day(overview_pred,filename):  
    sns.set_context('talk')
    overview_pred["Datum"] = pd.to_datetime(overview_pred["Datum"])
    overview_pred['Datum'] = overview_pred['Datum'].dt.date
    # Create a grouped bar chart, with job as the x-axis
    # and gender as the variable we're grouping on so there
    # are two bars per job.
    fig, ax = plt.subplots(figsize=(60, 20))
    # Our x-axis. We basically just want a list
    # of numbers from zero with a value for each
    # of our jobs.
    x = np.arange(len(overview_pred["Wochentag"]))
    # Define bar width. We need this to offset the second bar.
    bar_width = 0.4
    b1 = ax.bar(x, overview_pred["Bestellzahlen_Gesamt"], width=bar_width, label='Realwert')
    # Same thing, but offset the x.
    b2 = ax.bar(x + bar_width, overview_pred["pred"], width=bar_width, label='Prognose')
    # Fix the x-axes.
    ax.set_xticks(x + bar_width / 2)
    ax.set_xticklabels(overview_pred["Datum"])
    textstr = f"{overview_pred.Wochentag.iloc[1]}"
    # these are matplotlib.patch.Patch properties
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    # place a text box in upper left in axes coords
    ax.text(0.37, 0.99, textstr, transform=ax.transAxes, fontsize=25,
        verticalalignment='top', bbox=props)
    # Add legend.
    ax.legend(loc=1, prop={'size': 14})

    # Axis styling.
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_color('#DDDDDD')
    ax.tick_params(bottom=False, left=False)
    ax.set_axisbelow(True)
    ax.yaxis.grid(True, color='#EEEEEE')
    ax.xaxis.grid(False)

    # Add axis and chart labels.
    ax.set_xlabel('Wochentage', labelpad=15)
    ax.set_ylabel('Bestellzahlen', labelpad=15)
    #ax.set_title('Vergleich Prognose und Realwert', pad=15)

    # For each bar in the chart, add a text label.
    for bar in ax.patches:
        # The text annotation for each bar should be its height.
        bar_value = bar.get_height()
        # Format the text with commas to separate thousands. You can do
        # any type of formatting here though.
        text = f'{bar_value:,}'
        # This will give the middle of each bar on the x-axis.
        text_x = bar.get_x() + bar.get_width() / 2
        # get_y() is where the bar starts so we add the height to it.
        text_y = bar.get_y() + bar_value
        # If we want the text to be the same color as the bar, we can
        # get the color like so:
        bar_color = bar.get_facecolor()
        # If you want a consistent color, you can just set it as a constant, e.g. #222222
        ax.text(text_x, text_y, text, ha='center', va='bottom', color=bar_color,
          size=12)
    plt.savefig(filename, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()
    #plt.show
    #fig.tight_layout()

In [29]:
# Errors nahc tage vorhersage 
days = [pred_day1,pred_day2,pred_day3,pred_day4,pred_day5,pred_day6,pred_day7]

errors_multi_step = errors_nach_day_prediction(days,0,1)

weekdays = ["Montag","Dienstag","Mittwoch","Donnerstag","Freitag", "Samstag"]
weekday_array = []
for weekday in weekdays: 
    day = look_at_weekdays(days,weekday)
    weekday_array.append(day)
#rmse, mae, mape    
errors_weekday = errors_nach_day_prediction(weekday_array,3,4)
#mean, std, 05 quantile 
pred_numbers,real_numbers = numbers_nach_day_prediction(weekday_array,3,4)

create_folder(f"00_Report2_Graphs/barcharts")

chart_names = []
for n in range(0,len(weekday_array)):
    get_bar_per_day(weekday_array[n],f"00_Report2_Graphs/barcharts/{n}_{weekdays[n]}_bar.png")
    chart_names.append(f"00_Report2_Graphs/barcharts/{n}_{weekdays[n]}_bar.png")

In [40]:
pdf = PDF()
pdf = get_report2(chart_names,pdf)
pdf.output('Report_DPS.pdf', 'F')

''

In [39]:
def get_report2(chart_names, pdf):
    
    #df_prognose.Datum = pd.to_datetime(df_prognose.Datum).dt.date
    #Standard Größe für PDF 
    pdf_w=210
    pdf_h=297
    pdf.add_page()
    pdf.set_author(author= "Lukas Dech")
    pdf.set_xy(0, 0)
    pdf.set_font('arial', 'B', 14)
    pdf.cell(60)
    pdf.cell(10,14,"",0,2)
    pdf.cell(105, 10, "Bestellzahlen Vorhersage Alle Daten Reporting 2 DPS", 0, 2, 'C')
    pdf.cell(90, 3, " ", 0, 2, 'C')
    #pdf.set_font('arial',"", 12)
    #pdf.cell(105, 8, f"{min(df_prognose['Datum'])} - {max(df_prognose['Datum'])}", 1, 2, 'C')
    pdf.cell(90, 15, " ", 0, 2, 'C')
    
    pdf.cell(-15)
    pdf.cell(30, 10, 'Step', 1, 0, 'C')
    pdf.cell(30, 10, 'RMSE', 1, 0, 'C')
    pdf.cell(30, 10, 'MAE', 1, 0, 'C')
    pdf.cell(30, 10, 'MAPE', 1, 1, 'C')
    #pdf.set_x = 0
    pdf.cell(35)
    pdf.set_font('arial', '', 12)
    for i in range(0, len(errors_multi_step)):
        pdf.cell(30, 8, f"{i +1 }", 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(errors_multi_step[i][0])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(errors_multi_step[i][1])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(errors_multi_step[i][2])), 1, 1, 'C')
        pdf.cell(35)
    pdf.cell(90, 20, " ", 0, 2, 'C')
    pdf.set_font('arial', 'B', 14)
    
    #pdf.cell(-15)
    pdf.cell(30, 10, 'Wochentag', 1, 0, 'C')
    pdf.cell(30, 10, 'RMSE', 1, 0, 'C')
    pdf.cell(30, 10, 'MAE', 1, 0, 'C')
    pdf.cell(30, 10, 'MAPE', 1, 1, 'C')
    #pdf.set_x = 0
    pdf.cell(35)
    pdf.set_font('arial', '', 12)
    for i in range(0, len(pred_numbers)):
        pdf.cell(30, 8, f"{weekdays[i]}", 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(errors_weekday[i][0])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(errors_weekday[i][1])), 1, 0, 'C')
        pdf.cell(30, 8, '%s' % (str(errors_weekday[i][2])), 1, 1, 'C')
        pdf.cell(35)
    pdf.cell(90, 20, " ", 0, 2, 'C')
    pdf.set_font('arial', 'B', 14)
    pdf.cell(-35)
    pdf.cell(30, 10, 'Wochentag', 1, 0, 'C')
    pdf.cell(25, 10, 'Mean_r', 1, 0, 'C')
    pdf.cell(25, 10, 'Std_r', 1, 0, 'C')
    pdf.cell(25, 10, 'Q50_r', 1, 0, 'C')
    pdf.cell(10, 10, '', 0, 0, 'C')
    pdf.cell(25, 10, 'Mean_p', 1, 0, 'C')
    pdf.cell(25, 10, 'Std_p', 1, 0, 'C')
    pdf.cell(25, 10, 'Q50_p', 1, 1, 'C')
    #pdf.set_x = 0
    #pdf.cell(5)
    pdf.set_font('arial', '', 12)
    for i in range(0, len(real_numbers)):
        pdf.cell(30, 8, f"{weekdays[i]}", 1, 0, 'C')
        pdf.cell(25, 8, '%s' % (str(real_numbers[i][0])), 1, 0, 'C')
        pdf.cell(25, 8, '%s' % (str(real_numbers[i][1])), 1, 0, 'C')
        pdf.cell(25, 8, '%s' % (str(real_numbers[i][2])), 1, 0, 'C')
        pdf.cell(10, 8, "", 0, 0, 'C')
        pdf.cell(25, 8, '%s' % (str(pred_numbers[i][0])), 1, 0, 'C')
        pdf.cell(25, 8, '%s' % (str(pred_numbers[i][1])), 1, 0, 'C')
        pdf.cell(25, 8, '%s' % (str(pred_numbers[i][2])), 1, 1, 'C')
        #pdf.cell(5)
    pdf.cell(90, 20, " ", 0, 2, 'C')
    
    WIDTH = 200
    HEIGHT = 100
    for n in range(0,len(chart_names)):
        pdf.add_page(orientation = "L")
        y = pdf.get_y()
        x = pdf.get_x()
        pdf.image(chart_names[n], x = x + 5, y =  y + 5, w = pdf_w - 10 , h = pdf_h -10 , type = '', link = '')

    return pdf